# lembre-se de substituir /caminho/ pelo caminho do seu diretorio

In [ ]:
import scrapy
import time
import random
import sys
import re
import csv

from selenium import webdriver
from scrapy.http import HtmlResponse
from time import sleep
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from scrapy.selector import Selector

from urllib.parse import urlparse, parse_qs
from scrapy.crawler import CrawlerProcess


In [ ]:
'''
Processo para extrair os links para realizar web scraping dinamico no site
'''

class QuotesSpider(scrapy.Spider):
	name = "quotes"
	start_urls = ['https://www.doctoralia.com.br/pesquisa?q=Dermatologista&loc=Belo%20Horizonte%2C%20Minas%20Gerais%20MG&filters%5Bcalendar%5D%5B%5D=true&filters%5Bspecializations%5D%5B%5D=24&sorter=']
	a=0
	links_medicos=[]

	def parse(self, response):
		
		# Lógica para extrair links
		links = response.xpath('*//div/div/div/div[1]/div[1]/div/div[2]/h3/a/@href').extract()
		for link in links:
			self.links_medicos.append(link)
			
		# Lógica para verificar a próxima página
		next_page = response.xpath('//a[@data-test-id="pagination-next"]/@href').get()
		if next_page is not None:
			yield scrapy.Request(next_page, callback=self.parse)
			self.a += 1
		#remover links duplicados	
		else:
			
			self.links_medicos = set(self.links_medicos)
				
	#salvar links extraidos		
	def closed(self, reason):
		with open('/home/caminho/primeira_etapa_dados_medicos/site_medicos.csv', 'w', encoding='utf-8') as csvfile:
			for link in self.links_medicos:
			 	csvfile.write(link)
			 	csvfile.write("\n")

In [ ]:

extrair_sites = CrawlerProcess()
extrair_sites.crawl(QuotesSpider)
extrair_sites.start()

In [ ]:
#abrir selenium
options = webdriver.ChromeOptions() 
options.add_argument("user-data-dir=/home/caminho/Perfil")
options.add_argument("profile-directory=Profile 2")
driver = webdriver.Chrome(executable_path="/home/caminho/chromedriver", options=options)


In [ ]:
def buscar_agendas():
    agenda = driver.find_elements(By.XPATH, '//*[@id="object-profile"]//div[@class="card card-shadow-2"]')
    html_agenda_lista = []
    for elemento in agenda:
        html_agenda_lista.append(elemento.get_attribute('outerHTML'))
    return(html_agenda_lista)

def extrair_enderco():
    #atualiza os dados
    agenda = driver.find_elements(By.XPATH, '//*[@id="object-profile"]//div[@class="card card-shadow-2"]/div[2]')
    html_agenda_lista = []
    for elemento in agenda:
        html_agenda_lista.append(elemento.get_attribute('outerHTML'))
    html_agenda_str = ''.join(html_agenda_lista)

    selector = Selector(text=html_agenda_str)
    #extrai enderecos das agendas
    enderecos = selector.xpath('//div/h5/text()').extract()
    enderecos_normalizados = [linha.strip() for linha in enderecos]
    return(enderecos_normalizados)

def extrair_datas():
    #atualiza os dados
    agenda = driver.find_elements(By.XPATH, '//*[@id="object-profile"]//div[@class="card card-shadow-2"]/div[2]')
    html_agenda_lista_2 = []
    for elemento in agenda:
        html_agenda_lista_2.append(elemento.get_attribute('outerHTML'))
    html_agenda_str_2 = ''.join(html_agenda_lista_2)
    selector = Selector(text=html_agenda_str_2)
    #extrai datas
    datas = selector.xpath('//button[contains(@class, "calendar-slot")]/@aria-label').getall()
    return(datas)
    
def consulta_local():
    #analisa para ver se a consulta é local ou teleconsulta
    xpath = '//*[@id="object-profile"]//ul[@class="mb-2 nav nav-tabs nav-tabs-border-bottom"]/li[1]'
    clicar_consulta_local = driver.find_elements(By.XPATH,xpath)  
    if len(clicar_consulta_local) > 0 :
        clicar_consulta_local[0].click()

def def_nome_do_medico():
    try: 
        nome = driver.find_element(By.XPATH, '//*[@id="object-profile"]//div[@class="unified-doctor-header-info__name"]/span[@itemprop="name"]')
        nome = nome.text
    except Exception as e:
        nome = 'nao_encontrado'
    return(nome)

def crm_medico():
    try: 
        crm =  driver.find_element(By.XPATH,'//div[@class="text-muted small"]')
        crm = crm.text
        crm = re.findall(r'\b\d+\b', crm)
        crm = crm[0]
    except Exception as e:
        crm = ''
    return(crm)

def outros_enderecos_no_site():
    novos_enderecos = driver.find_elements(By.XPATH, '//*[@id="object-profile"]//div[@itemprop="address"]')
    html_novos_enderecos_lista = []
    for elemento in novos_enderecos:
        html_novos_enderecos_lista.append(elemento.get_attribute('outerHTML'))
    html_novos_enderecos_str = ''.join(html_novos_enderecos_lista)

    selector2 = Selector(text=html_novos_enderecos_str)
    outra_forma_enderecos = list(set(selector2.xpath('//span[@data-test-id="address-info-street"]/text()').getall()))
    return(outra_forma_enderecos)

In [ ]:
#abrir arquivos
nome_arquivo = '/home/caminho/primeira_etapa_dados_medicos/site_medicos.csv'
url_site_medico = []
with open(nome_arquivo, 'r') as arquivo_csv:
    for line in arquivo_csv:
        line_limpa = line.strip()
        url_site_medico.append(line_limpa)
cainho_arquivo_datas = "/home/caminho/primeira_etapa_dados_medicos/dados_medicos_data.csv"
cainho_arquivo_selenium = "/home/caminho/primeira_etapa_dados_medicos/dados_medicos_selenium.csv"
arquivo_datas = open(cainho_arquivo_datas, 'w+')
arquivo_selenium = open(cainho_arquivo_selenium, 'w+')


In [ ]:
arquivo_datas = open(cainho_arquivo_datas, 'a+')
arquivo_selenium = open(cainho_arquivo_selenium, 'a+')

total = len(url_site_medico)
atual = 0
passos = 0
for url_entrar in url_site_medico:
    driver.get(url_entrar)
    time.sleep(10)
    html_agenda_lista = buscar_agendas()
    nome_do_medico = def_nome_do_medico()
    crm = crm_medico()
    consulta_local()
    time.sleep(5)
    #html_agenda_lista = buscar_agendas()
    endereco = extrair_enderco()
    time.sleep(2)
    passos = passos +1
    for i in range(len(endereco)):
        #clica nos botoes de baixar aba de endereço
        try:
            clicar = driver.find_elements(By.XPATH, '//*[@id="object-profile"]//div[@class="multiselect dp-multiselect multiselect-lg"]/div')
            clicar[0].click()
            time.sleep(2)
        except Exception as e:
            print(f'erro medico {nome_do_medico} no endereco {endereco[i]}')

        
        xpath = f'//*[@id="object-profile"]//div[@class="w-100 option-container"]//ul[@class="multiselect__content"]/li[{i + 1}]/span/div/h5'
        try:
            clicar_endereco = driver.find_elements(By.XPATH,xpath)
            clicar_endereco[0].click()
            time.sleep(2)

        except Exception as e:
            print(f'erro medico {nome_do_medico} no endereco {endereco[i]}')
        
        datas = extrair_datas()
        escrever = f'{nome_do_medico};{endereco[i]};{datas};{crm};{url_entrar}'
        arquivo_datas.write(escrever)
        arquivo_datas.write('\n')
       
    #pega os enderecos no site independente da agenda
    endereco_fora_agenda = outros_enderecos_no_site()
    for i in range(len(endereco_fora_agenda)):
        escrever = f'{nome_do_medico};{endereco_fora_agenda[i]}; {crm};{url_entrar}'
        arquivo_selenium.write(escrever)
        arquivo_selenium.write('\n')
    atual = atual + 1
    print(f'final {atual} de {total}')
    if passos > 20 :
        arquivo_selenium.close()
        arquivo_datas.close()
        arquivo_datas = open(cainho_arquivo_datas, 'a+')
        arquivo_selenium = open(cainho_arquivo_selenium, 'a+')
        passos = 0
        
arquivo_selenium.close()
arquivo_datas.close()